In [1]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, sortdict, data, quantify, atom, atoms, list_atoms, list_multiply, mapt, Node
from aocutils.grid import TURNRIGHT, TURNLEFT, move, gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, ranges2grid, manhattan, conv1d, conv2d, rotate, rot90
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0)
winds = [N, E, S, W]
inf = float('inf')

In [2]:
   
@cache
def direcnext(start, goal):
    # goes to a button and back to A
    
    keys = {'^':(0,1), 'A':(0,2), '<':(1,0), 'v':(1,1), '>':(1,2)}
    
    # print('pressing from', loc, 'goal', keys[char])
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    options = []
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))
        
        # for waytoA in nx.all_shortest_paths(G, keys[goal], keys['A']):
            # options.append(path + ['A'] + waytoA[1:])
    return options

@cache 
def numericnext(start, goal):
    keys = {'7':(0,0), '8':(0,1), '9':(0,2), '4':(1,0), '5':(1,1), '6': (1,2), '1':(2,0), '2':(2,1), '3':(2,2), '0':(3,1), 'A':(3,2)}
    # gives back options to next step
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))

def numerictotal(goalstring):
    # given a string, comes up with paths how to type in that string
    paths = [[]]
    for a,b in zippify('A'+goalstring, 2):
        newpaths = []
        for p1 in paths:
            for p2 in numericnext(a,b):
                newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return tuple(paths)
def directtotal(goalstring):
    # given instructions for a directional robot, comes up with paths how to type in that string
    paths = [[]]
    
    for a,b in zippify(['A']+goalstring, 2):
        newpaths = []
        if a == b:
            for p1 in paths:
                newpaths.append(p1.copy() + ['press'])
        else:
            for p2 in direcnext(a,b):
                # print('from to', a,b,'path', p2)
                for p1 in paths:
                    newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return paths
            
        
def numericpath_to_direcpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (3,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)

def direcpath_to_directpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (0,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)
 


In [4]:
mapping = {}
def parse(ins, ins2):
    global mapping
    cur = 'A'
    p2 = 0
    for ch in ins:
        if cur == ch:
            newp2 = p2
            mapping[(cur, ch)] = ['A']
        else:
            newp2 = ins2.index('A', p2+1)
            new = ins2[p2:newp2+1]
            if (cur,ch) in mapping and mapping[(cur, ch)] != new:
                print('value already in from', cur, 'to', ch, new, '!=', mapping[(cur, ch)])
            mapping[(cur, ch)] = new
        # print(cur, ch,ins2[p2:newp2+1], newp2)
        p2 = newp2+1
        cur = ch
        
parse(list('<A^A>^^AvvvA'),list('v<<A>>^A<A>AvA<^AA>A<vAAA>^A'))
mapping


# this code is to fill the mapping that is used later on
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

mapping = {}
answers = defaultdict(lambda: defaultdict(lambda: int(99999999)))
for comb in list(product(list('<>^vA'), repeat = 2)):
    print(comb)
    
    nextlevel = directtotal(list(comb))
    for option in nextlevel:
        ins1 = direcpath_to_directpath(option)
        print(ins1)
        for option2 in directtotal(ins1):
            ins2 = direcpath_to_directpath(option2)
            for option3 in directtotal(ins2):
                ins3 = direcpath_to_directpath(option3)
                answers[comb][tuple(ins1)] = min(answers[comb][tuple(option)], len(ins3))
                # for option4 in directtotal(ins3):
                #     ins4 = direcpath_to_directpath(option4)
for c in answers:
    best = min(v for k,v in answers[c].items())
    cnt = sum(1 for k,v in answers[c].items() if v == best)
    candidates = [k for k,v in answers[c].items() if v == best]
    parse(c, candidates[-1])
    


('<', '<')
['<', 'v', '<', 'A', 'A']
['v', '<', '<', 'A', 'A']
('<', '>')
['<', 'v', '<', 'A', '>', '>', 'A']
['v', '<', '<', 'A', '>', '>', 'A']
('<', '^')
['<', 'v', '<', 'A', '>', '^', 'A']
['v', '<', '<', 'A', '>', '^', 'A']
('<', 'v')
['<', 'v', '<', 'A', '>', 'A']
['v', '<', '<', 'A', '>', 'A']
('<', 'A')
['<', 'v', '<', 'A', '>', '^', '>', 'A']
['v', '<', '<', 'A', '>', '^', '>', 'A']
['<', 'v', '<', 'A', '>', '>', '^', 'A']
['v', '<', '<', 'A', '>', '>', '^', 'A']
('>', '<')
['v', 'A', '<', '<', 'A']
('>', '>')
['v', 'A', 'A']
('>', '^')
['v', 'A', '^', '<', 'A']
['v', 'A', '<', '^', 'A']
('>', 'v')
['v', 'A', '<', 'A']
('>', 'A')
['v', 'A', '^', 'A']
('^', '<')
['<', 'A', 'v', '<', 'A']
('^', '>')
['<', 'A', '>', 'v', 'A']
['<', 'A', 'v', '>', 'A']
('^', '^')
['<', 'A', 'A']
('^', 'v')
['<', 'A', 'v', 'A']
('^', 'A')
['<', 'A', '>', 'A']
('v', '<')
['<', 'v', 'A', '<', 'A']
['v', '<', 'A', '<', 'A']
('v', '>')
['<', 'v', 'A', '>', 'A']
['v', '<', 'A', '>', 'A']
('v', '^')
['<'

In [725]:
# these are the keypresses robot 1 needs to do
# works on example but not on 25 robots
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

@cache
def explode(smallpart):
    i = smallpart
    smallpart = flatten([mapping[a] for a in zippify((smallpart))])
    for step in range(14):
        smallpart = flatten([mapping[a] for a in zippify(['A']+list(smallpart))])
    
    return len(smallpart)

ans = 0
for line in lines:
    minn = inf
    for p0 in numerictotal(line):
        ins = numericpath_to_direcpath(p0)
        for step in range(10):
            ins = flatten([mapping[a] for a in zippify(['A']+ins,2)])
        # minn = min(len(ins), minn)

        temp = 0
        for smallpart in zippify(['A']+ins,2):
            temp += explode(smallpart)
        minn = min(minn, temp)
            
    print('found', minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

found 99882977150 463
found 95771967742 340
found 102689071222 129
found 91715319286 083
found 103671559318 341


135019550868544

('<', '<')
['<', 'v', '<', 'A', 'A']
['v', '<', '<', 'A', 'A']
('<', '>')
['<', 'v', '<', 'A', '>', '>', 'A']
['v', '<', '<', 'A', '>', '>', 'A']
('<', '^')
['<', 'v', '<', 'A', '>', '^', 'A']
['v', '<', '<', 'A', '>', '^', 'A']
('<', 'v')
['<', 'v', '<', 'A', '>', 'A']
['v', '<', '<', 'A', '>', 'A']
('<', 'A')
['<', 'v', '<', 'A', '>', '^', '>', 'A']
['v', '<', '<', 'A', '>', '^', '>', 'A']
['<', 'v', '<', 'A', '>', '>', '^', 'A']
['v', '<', '<', 'A', '>', '>', '^', 'A']
('>', '<')
['v', 'A', '<', '<', 'A']
('>', '>')
['v', 'A', 'A']
('>', '^')
['v', 'A', '^', '<', 'A']
['v', 'A', '<', '^', 'A']
('>', 'v')
['v', 'A', '<', 'A']
('>', 'A')
['v', 'A', '^', 'A']
('^', '<')
['<', 'A', 'v', '<', 'A']
('^', '>')
['<', 'A', '>', 'v', 'A']
['<', 'A', 'v', '>', 'A']
('^', '^')
['<', 'A', 'A']
('^', 'v')
['<', 'A', 'v', 'A']
('^', 'A')
['<', 'A', '>', 'A']
('v', '<')
['<', 'v', 'A', '<', 'A']
['v', '<', 'A', '<', 'A']
('v', '>')
['<', 'v', 'A', '>', 'A']
['v', '<', 'A', '>', 'A']
('v', '^')
['<'

In [719]:
mapping

{('A', '<'): ('v', '<', '<', 'A'),
 ('<', '<'): ['A'],
 ('<', '>'): ('>', '>', 'A'),
 ('<', '^'): ('>', '^', 'A'),
 ('<', 'v'): ('>', 'A'),
 ('<', 'A'): ('>', '>', '^', 'A'),
 ('A', '>'): ('v', 'A'),
 ('>', '<'): ('<', '<', 'A'),
 ('>', '>'): ['A'],
 ('>', '^'): ('<', '^', 'A'),
 ('>', 'v'): ('<', 'A'),
 ('>', 'A'): ('^', 'A'),
 ('A', '^'): ('<', 'A'),
 ('^', '<'): ('v', '<', 'A'),
 ('^', '>'): ('v', '>', 'A'),
 ('^', '^'): ['A'],
 ('^', 'v'): ('v', 'A'),
 ('^', 'A'): ('>', 'A'),
 ('A', 'v'): ('<', 'v', 'A'),
 ('v', '<'): ('<', 'A'),
 ('v', '>'): ('>', 'A'),
 ('v', '^'): ('^', 'A'),
 ('v', 'v'): ['A'],
 ('v', 'A'): ('^', '>', 'A'),
 ('A', 'A'): ['A']}

In [717]:
answers[('v', 'A')]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {((0, 1), (1, 1), 'press', (0, 1), (0, 2), 'press'): 99999999,
             ('<', 'v', 'A', '^', '>', 'A'): 40,
             ((1, 2), (1, 1), 'press', (0, 1), (0, 2), 'press'): 99999999,
             ('v', '<', 'A', '^', '>', 'A'): 44,
             ((0, 1), (1, 1), 'press', (1, 2), (0, 2), 'press'): 99999999,
             ('<', 'v', 'A', '>', '^', 'A'): 40,
             ((1, 2), (1, 1), 'press', (1, 2), (0, 2), 'press'): 99999999,
             ('v', '<', 'A', '>', '^', 'A'): 44})

In [712]:
# the correct one
mapping

{('A', '<'): ['v', '<', '<', 'A'],
 ('<', 'A'): ['>', '>', '^', 'A'],
 ('A', '^'): ['<', 'A'],
 ('^', 'A'): ['>', 'A'],
 ('^', '^'): ['A'],
 ('^', '>'): ['v', '>', 'A'],
 ('>', 'A'): ['^', 'A'],
 ('A', 'v'): ['v', '<', 'A'],
 ('v', 'v'): ['A'],
 ('v', 'A'): ['>', '^', 'A'],
 ('>', '^'): ['<', '^', 'A'],
 ('A', '>'): ['v', 'A'],
 ('^', '<'): ['v', '<', 'A'],
 ('<', '<'): ['A'],
 ('>', '>'): ['A'],
 ('<', '^'): ['>', '^', 'A']}

In [711]:
# this code is to fill the mapping that is used later on
f = open("t0.txt").read().split("\n")
lines = [line for line in f]
lines[0]

mapping = {}
ans = 0
for line in lines:
    minn = inf
    
    for p0 in numerictotal(line):
        
        ins = numericpath_to_direcpath(p0)
        print(ins)
        
        options = directtotal(ins)
        best = min(len(o) for o in options)
        for p1 in options:
            if len(p1) == best:
                ins2 = direcpath_to_directpath(p1)
                parse(ins, ins2)
                o2 = directtotal(ins2)
                besto2 = min(len(o) for o in o2)
                for p2 in o2:
                    if len(p2) == besto2:
                        p3 = direcpath_to_directpath(p2)
                        # parse(ins2,p3)
                        minn = min(len(p2), minn)
    print(minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

['<', 'A', '^', 'A', '^', '^', '>', 'A', 'v', 'v', 'v', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '>', '^', 'A'] != ['>', '^', '>', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '^', '>', 'A'] != ['>', '>', '^', 'A']
value already in from ^ to > ['v', '>', 'A'] != ['>', 'v', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '>', '^', 'A'] != ['>', '^', '>', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '^', '>', 'A'] != ['>', '>

126384

In [439]:
2**25

33554432

In [434]:
### THIS WORKS
   
@cache
def direcnext(start, goal):
    # goes to a button and back to A
    
    keys = {'^':(0,1), 'A':(0,2), '<':(1,0), 'v':(1,1), '>':(1,2)}
    
    # print('pressing from', loc, 'goal', keys[char])
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    options = []
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))
        
        # for waytoA in nx.all_shortest_paths(G, keys[goal], keys['A']):
            # options.append(path + ['A'] + waytoA[1:])
    return options

@cache 
def numericnext(start, goal):
    keys = {'7':(0,0), '8':(0,1), '9':(0,2), '4':(1,0), '5':(1,1), '6': (1,2), '1':(2,0), '2':(2,1), '3':(2,2), '0':(3,1), 'A':(3,2)}
    # gives back options to next step
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))
def numerictotal(goalstring):
    # given a string, comes up with paths how to type in that string
    paths = [[]]
    for a,b in zippify('A'+goalstring, 2):
        newpaths = []
        for p1 in paths:
            for p2 in numericnext(a,b):
                newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return tuple(paths)
def directtotal(goalstring):
    # given instructions for a directional robot, comes up with paths how to type in that string
    paths = [[]]
    
    for a,b in zippify(['A']+goalstring, 2):
        newpaths = []
        if a == b:
            for p1 in paths:
                newpaths.append(p1.copy() + ['press'])
        else:
            for p2 in direcnext(a,b):
                # print('from to', a,b,'path', p2)
                for p1 in paths:
                    newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return paths
            
        
def numericpath_to_direcpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (3,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)

def direcpath_to_directpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (0,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)
 


State = namedtuple('State', ['pad0', 'pad1', 'pad2', 'pad3', 'out'])
goal = '029A'


In [435]:
2**25

33554432

In [436]:
# these are the keypresses robot 1 needs to do
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]
ans = 0
for line in lines:
    minn = inf
    
    for p0 in numerictotal(line):
        
        ins = numericpath_to_direcpath(p0)
        
        for p1 in directtotal(ins):
            # print(p1)
            ins2 = direcpath_to_directpath(p1)
            for p2 in directtotal(ins2):
                minn = min(len(p2), minn)
    print(minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

KeyboardInterrupt: 

In [ ]:
# these are the keypresses robot 1 needs to do
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]
ans = 0
for line in lines:
    minn = inf
    for p0 in numerictotal(line):
        ins = numericpath_to_direcpath(p0)
        for p1 in directtotal(ins):
            # print(p1)
            ins2 = direcpath_to_directpath(p1)
            for p2 in directtotal(ins2):
                minn = min(len(p2), minn)
    print(minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

In [353]:
def direcpath_to_directpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return needed


['<', 'A', '^', 'A', '>', '^', '^', 'A', 'v', 'v', 'v', 'A']

In [376]:
minn = inf
for i in directtotal(ins):
    newins = direcpath_to_directpath(i)
    # print(len (newins), newins)
    for p2 in directtotal(newins):
        minn = min(len(p2), minn)
minn
    

68

In [60]:
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0)

idx = 0
cur = State((3,2),(0,2),(0,2),(0,2),'')
for path in (numericnext((3,2), goal[idx])):
    # get options for paths for first robot
    # these are the goals for the next robot
    keypresses = numericpath_to_direcpath(path)
    print('path for initial robot is', path, 'with these keypresses', keypresses)
    for keypress in keypresses:
        shortestpathstopressbutton = direcnext((0,2), keypress)
        print(shortest)
    idx1 = 0
    # while idx1 < len(keypresses):
        
    #     for in direcnext(o):

path for initial robot is [(3, 2), (3, 1)] with these keypresses [(0, -1)]
pressing from (0, 2) goal (1, 0)
[[(0, 2), (0, 1), (1, 1), (1, 0), (1, 0), (1, 1), (0, 1), (0, 2)], [(0, 2), (0, 1), (1, 1), (1, 0), (1, 0), (1, 1), (1, 2), (0, 2)], [(0, 2), (1, 2), (1, 1), (1, 0), (1, 0), (1, 1), (0, 1), (0, 2)], [(0, 2), (1, 2), (1, 1), (1, 0), (1, 0), (1, 1), (1, 2), (0, 2)]]
<generator object _build_paths_from_predecessors at 0x000001460E3CD470>


In [ ]:
# grid
grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])
(minr, minc), (maxr, maxc), (rows, cols) = dimensions(grid)


In [ ]:
# graph
conn = graphparse("in.txt", parser=lambda line: [x for x in line.split()])
conn = graphparse(
    "in.txt",
    parser=lambda line: [int(x) if x.isnumeric() else x for x in line.split()],
    weights=True,
)

In [7]:
E = 1; W = -1; N = 1j; S = -1j
TURNLEFT = lambda x: x * 1j
TURNRIGHT = lambda x: x * -1j

# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}